## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

# Reading Data

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fake_news_classifier/train.csv' )

Mounted at /content/drive


# EDA

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.9.2'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
### Vocabulary size
voc_size=5000

# Dataset Preprocessing 

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
messages.reset_index(inplace=True)

In [21]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

# Onehot Representation

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3620, 4320, 762, 1760, 1807, 1904, 1163, 4698, 2011, 1021],
 [2278, 1415, 2410, 4260, 4001, 64, 3349],
 [2800, 1289, 1750, 1682],
 [938, 2651, 3961, 1440, 4000, 3948],
 [889, 4001, 1552, 4790, 2396, 1648, 4001, 990, 1657, 4433],
 [4461,
  2857,
  1338,
  206,
  2485,
  1323,
  3954,
  3991,
  3442,
  2427,
  1176,
  1677,
  3830,
  4474,
  3349],
 [4842, 3267, 1172, 960, 2046, 615, 883, 1676, 1435, 1387, 3351],
 [3945, 6, 3524, 255, 2687, 2916, 1323, 2317, 1435, 1387, 3351],
 [3137, 412, 3928, 466, 3103, 2150, 1549, 4775, 1323, 4971],
 [2601, 1824, 1969, 4168, 34, 3502, 4350, 1573],
 [3378, 4285, 339, 3269, 4605, 1518, 2708, 2565, 3283, 3439, 3332],
 [1440, 2942, 1807, 2150, 1323, 2687],
 [1325, 4941, 3449, 4563, 1515, 1440, 2680, 1097, 770],
 [247, 3703, 1870, 3271, 2868, 1841, 261, 1435, 1387, 3351],
 [3927, 981, 4809, 4538, 927, 1435, 1387, 3351],
 [1158, 2997, 2028, 925, 3212, 3386, 195, 1668, 1944, 1840],
 [2058, 2358, 1415],
 [224, 4416, 645, 835, 1323, 4546, 4971, 3349],
 [302

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr[1]

[2278, 1415, 2410, 4260, 4001, 64, 3349]

### Embedding Representation

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3620 4320  762 ...    0    0    0]
 [2278 1415 2410 ...    0    0    0]
 [2800 1289 1750 ...    0    0    0]
 ...
 [3619 1870 4349 ...    0    0    0]
 [4805 2687 2433 ...    0    0    0]
 [ 755 2805 1317 ...    0    0    0]]


In [31]:
embedded_docs[1]

array([2278, 1415, 2410, 4260, 4001,   64, 3349,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [32]:
embedded_docs[0]

array([3620, 4320,  762, 1760, 1807, 1904, 1163, 4698, 2011, 1021,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

# Define the model SimpleRNN

In [33]:
# Define the model
from keras.layers import Embedding, SimpleRNN, Dense
embedding_vector_features=40 ##features representation
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SimpleRNN(32))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2336      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 202,369
Trainable params: 202,369
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
len(embedded_docs),y.shape

(18285, (18285,))

# train_test_split

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [38]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 10ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/10
192/192 [======

# Performance Metrics And Accuracy

In [39]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5666942833471417

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       1.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.78      0.50      0.36      6035
weighted avg       0.75      0.57      0.41      6035

